# 2 задача
## Найти решение краевой задачи для одномерного стационарного уравнения теплопроводности

### Постановка задачи (вариант №1 задание №4)

#### Уравнение теплопроводности:

\begin{gather*}
   {d \over dx} [k(x){du \over dx}] - q(x)u = -f(x)\\
    k(x) = sin^2(x) + 1;\; \; \;
    q(x) = sin(x); \; \; \;
    f(x) = e^x
\end{gather*}

\begin{equation}
0 \leq x \leq 1 
\end{equation}

#### Краевые условия задачи:

\begin{equation}
\left.
  \begin{array}{ccc}
      k(0) u_x'(0) = u(0) - 1 \\
      -k(1) u_x'(1) = u(1) - 1
  \end{array}
\right\}
\end{equation}


In [46]:
u0, u1 = 0, 1            # концы отрезка
L = 20481                 # число узлов
h = (u1 - u0) / (L - 1)  # шаг сетки

delta1, delta2 = 1, 1
eps1, eps2 = 1, 1

### Заменим производные в узлах сетки конечными разностями

\begin{equation}
\left\{
  \begin{array}{ccc}
      \frac{k_{l+1/2} \: (u_{l+1}-u_l) - k_{l-1/2} \: (u_l-u_{l-1})}{h^2} + q_{l} u_l = -f_l, \;\;\;\;\; l \in [1;L-1]\\
      k_0 {u_1-u_0 \over h} = \delta_1 u_0 - \varepsilon_1 \\ 
      -k_L {u_L-u_{L-1} \over h} = \delta_2 u_L - \varepsilon_2
  \end{array}
\right.
\end{equation}

#### Теперь необходимо решить полученную систему линейных уравнений

Перепишем систему в более удобном виде:

\begin{equation} 
\left\{
  \begin{array}{ccc}
      k_{l-1/2} \: u_{l-1} + (h^2 q_l - k_{l-1/2} - k_{l+1/2}) \: u_l + k_{l+1/2} \: u_{l+1} = -h^2 f_l, \;\;\;\;\; l \in [1;L-1]\\
      (k_0 + h \delta_1)\:u_0 - k_0 u_1  = h \varepsilon_1 \\ 
      -k_L u_{L-1} + (k_L + h \delta_2)\:u_L = h \varepsilon_2\\
  \end{array}
\right.
\end{equation}

Здесь разумно сделать замену переменных в уравнениях системы:

\begin{align*}
a_0& = 0           &  b_0 & = k_0 + h \delta_1                 &  c_0& = -k_0          &   d_0 & = h \varepsilon_1\\
a_l& = k_{l-1/2}   &  b_l&  = h^2 q_l - k_{l-1/2} - k_{l+1/2}  &  c_l& = k_{l+1/2}     &   d_l & = -h^2 f_l \\
a_L& = -k_L        &  b_L&  =k _L + h \delta_2                 &  c_L& = 0             &   d_L & = h \varepsilon_2
\end{align*}

Таким образом, система алгебраических уравнений примет вид:

\begin{equation} 
a_l u_{l-1} + b_l u_l + c_l u_{l+1} = d_l, \; \; \; l \in [0, L]
\end{equation}

Это матрица трёхдиагонального вида. Для таких матриц используется метод прогонки (алгоритм Томаса). Мы же воспользуемся встроенной функцией из numpy.linalg для разреженной матрицы из scipy.sparse

In [47]:
from math import *
from scipy.sparse import dia_matrix
#from numpy import linalg as LA
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

k = lambda x: sin(x)**2 + 1
q = lambda x: sin(x)
f = lambda x: exp(x)

a0, b0, c0, d0 = 0, k(0) + h*delta1, -k(0), h*eps1
aL, bL, cL, dL = -k(1), k(1) + h*delta2, 0, h*eps2

a = lambda x: k(x - h/2)
b = lambda x: h**2 * q(x) - k(x - h/2) - k(x + h/2)
c = lambda x: k(x + h/2)
d = lambda x: -h**2 * f(x)

In [48]:
upper_arr = [0, c0] + [c(l*h) for l in range(1, L-1)] 
middle_arr = [b0] + [b(l*h) for l in range(1, L-1)] + [bL]
lower_arr = [a(l*h) for l in range(1, L-1)] + [aL, 0]

# Создадим матрицу системы:
A = dia_matrix((np.array([lower_arr, middle_arr, upper_arr]), [-1, 0, 1]), shape=(L, L)).toarray()

# Создадим вектор свободных членов:
B = np.array([d0] + [d(l*h) for l in range(1, L-1)] + [dL])

# Получим решение:
solution = np.linalg.solve(A, B) 

In [5]:
sol32 = solution[::32]

In [9]:
sol64 = solution[::64]

In [26]:
sol128 = solution[::128]

In [34]:
sol256 = solution[::256]

In [39]:
sol512 = solution[::512]

In [44]:
sol1024 = solution[::1024]

In [49]:
sol2048 = solution[::2048]

In [50]:
# решения в 11 точках на [0;1]
pd.DataFrame([sol32,sol64,sol128,sol256,sol512,sol1024,sol2048],columns=(np.linspace(0,1,num=11)),index=[321,641,1281,2561,5121,10241,20481])

,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
321,2.345483,2.474199,2.587215,2.679783,2.748567,2.791710,2.808579,2.799377,2.764741,2.705413,2.621996
641,2.347702,2.476560,2.589714,2.682409,2.751308,2.794549,2.811499,2.802361,2.767772,2.708473,2.625069
1281,2.348812,2.477741,2.590963,2.683722,2.752678,2.795969,2.812959,2.803853,2.769287,2.710004,2.626606
2561,2.349366,2.478331,2.591588,2.684379,2.753364,2.796678,2.813690,2.804599,2.770045,2.710769,2.627375
5121,2.349643,2.478626,2.591900,2.684707,2.753706,2.797033,2.814055,2.804973,2.770424,2.711152,2.627759
10241,2.349782,2.478774,2.592056,2.684871,2.753878,2.797211,2.814238,2.805159,2.770614,2.711343,2.627952
20481,2.349851,2.478847,2.592134,2.684953,2.753963,2.797300,2.814329,2.805253,2.770709,2.711439,2.628048


In [53]:
err32 = abs(sol32-sol2048)/sol2048
err64 = abs(sol64-sol2048)/sol2048
err128 = abs(sol128-sol2048)/sol2048
err256 = abs(sol256-sol2048)/sol2048
err512 = abs(sol512-sol2048)/sol2048
err1024 = abs(sol1024-sol2048)/sol2048

In [54]:
# относительная ошибка 
pd.DataFrame([err32,err64,err128,err256,err512,err1024],columns=(np.linspace(0,1,num=11)),index=[321,641,1281,2561,5121,10241])

,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
321,0.001859,0.001875,0.001898,0.001926,0.001959,0.001998,0.002043,0.002095,0.002154,0.002222,0.002303
641,0.000915,0.000923,0.000934,0.000948,0.000964,0.000983,0.001006,0.001031,0.001060,0.001094,0.001133
1281,0.000442,0.000446,0.000452,0.000459,0.000467,0.000476,0.000487,0.000499,0.000513,0.000529,0.000549
2561,0.000206,0.000208,0.000211,0.000214,0.000218,0.000222,0.000227,0.000233,0.000239,0.000247,0.000256
5121,0.000088,0.000089,0.000090,0.000092,0.000093,0.000095,0.000097,0.000100,0.000103,0.000106,0.000110
10241,0.000029,0.000030,0.000030,0.000031,0.000031,0.000032,0.000032,0.000033,0.000034,0.000035,0.000037
